In [1]:
import numpy as np
import py_scripts.processingIO as pio
import pandas as pd

from pathlib import Path
from py_scripts.forceClass import Forces
from py_scripts.forceBinsClass import ForceBins

In [2]:
data_location = r'./'
force_flag = 1
forceBin_flag = 1

force_paths = pio.get_files(data_location, 'force_0.dat')
force_paths.sort()

forceBins_paths = pio.get_files(data_location, 'forceBin_0.dat')
forceBins_paths.sort()

In [3]:
total_cycles = 20
number_cycles = 20
if force_flag == 1:
    forces = [Forces(force_path, number_cycles, total_cycles, True, True, filterWindow = 101) for force_path in force_paths]
if forceBin_flag == 1:
    forceBins = [ForceBins(forceBin_path, number_cycles, total_cycles, True, True) for forceBin_path in forceBins_paths]

In [4]:
density = 1026.021
U = 1.0
A = 1.0

for force in forces:
    force.toCoefficients(density, U, A)

In [5]:
save_location = Path(r'Output_data/')
if ~save_location.joinpath('forces').is_dir():
    Path.mkdir(save_location.joinpath('forces'), parents=True, exist_ok=True)
    Path.mkdir(save_location.joinpath('force_bins'),  parents=True, exist_ok=True)

In [6]:
if force_flag == 1:    
    for i, force in enumerate(forces):
        df = pd.DataFrame({'time':force.filteredForces['time'], 'total_x':force.filteredForces['total']['x'], 'total_y':force.filteredForces['total']['y'], 
            'total_z':force.filteredForces['total']['z'], 'pressure_x':force.filteredForces['pressure']['x'], 'pressure_y':force.filteredForces['pressure']['y'], 
            'pressure_z':force.filteredForces['pressure']['z'], 'viscous_x':force.filteredForces['viscous']['x'], 'viscous_y':force.filteredForces['viscous']['y'], 
            'viscous_z':force.filteredForces['viscous']['z'],'CD':force.Cd, 'CDf':force.Cdf, 'CDp':force.Cdp, 'CL':force.Cl})
        save_file = force_paths[i].parts[1] + '_' + force_paths[i].parts[2] + '.csv'

        df.to_csv(save_location.joinpath('forces', save_file), index=False)


In [19]:
if forceBin_flag == 1:    
    for i, forceBin in enumerate(forceBins):
        number_bins = forceBin.bins
        x_coord = forceBin.forceCoord_x
        y_coord = forceBin.forceCoord_y
        z_coord = forceBin.forceCoord_z
        dfs = []
        for bin in np.arange(0, number_bins, 1):
            dfs.append(pd.DataFrame({'total_x':forceBin.filteredForces[bin]['total']['x'], 'total_y':forceBin.filteredForces[bin]['total']['y'], 
                'total_z':forceBin.filteredForces[bin]['total']['z'], 'pressure_x':forceBin.filteredForces[bin]['pressure']['x'], 
                'pressure_y':forceBin.filteredForces[bin]['pressure']['y'], 'pressure_z':forceBin.filteredForces[bin]['pressure']['z'], 
                'viscous_x':forceBin.filteredForces[bin]['viscous']['x'], 'viscous_y':forceBin.filteredForces[bin]['viscous']['y'], 
                'viscous_z':forceBin.filteredForces[bin]['viscous']['z']}))

        base_df = pd.concat(dfs, axis=1)
        base_df.insert(loc=0, column='time', value=forceBin.filteredForces['time'])

        save_file = forceBins_paths[i].parts[1] + '_' + forceBins_paths[i].parts[2] + '.csv'

        with open(save_location.joinpath('force_bins', save_file), 'w') as f:
            f.write('# ' + 'No Bins' + str(number_bins) + '\n')
            f.write('# x-coordinate, ' + np.array2string(x_coord, separator=',', formatter={'float_kind':lambda x: "%.4f" % x}).replace('[', '').replace(']','').replace('\n',''))
            f.write('\n')
            f.write('# y-coordinate, ' + np.array2string(y_coord, separator=',', formatter={'float_kind':lambda x: "%.4f" % x}).replace('[', '').replace(']','').replace('\n',''))
            f.write('\n')
            f.write('# z-coordinate, ' + np.array2string(z_coord, separator=',', formatter={'float_kind':lambda x: "%.4f" % x}).replace('[', '').replace(']','').replace('\n',''))
            f.write('\n')
            f.write('\n')

            f.close()

        base_df.to_csv(save_location.joinpath('force_bins', save_file), mode='a', index=False)